<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_ShallowNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotinas de Treinamento


## Configurações Iniciais 

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import pandas as pd
import numpy as np
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## Lendo a Base

In [72]:
# Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"

# Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA = "/content/drive/MyDrive/TCC/dados/processadas/Base1JuntaHateENotHateDaLeila.csv"

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA = "/content/drive/MyDrive/TCC/dados/processadas/Base2_maior4_menor25_limpissima.csv"

In [73]:
#trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA, index_col=0)
trainDF = pd.read_csv(PATH_BASE1_JUNTO_COM_DA_LEILA, index_col=0)

In [74]:
# Agrupa dados por label
trainDF.groupby('label').count()

,text
label,
hate,8017
hathate,1
not_hate,83465


In [51]:
categorias_desejadas = ['hate', 'not_hate'] # !!!
df_filtrado = trainDF[trainDF.label.isin(categorias_desejadas)]
df_filtrado.shape

(91482, 3)

In [52]:
df_filtrado

,Unnamed: 0,text,label
0,1,retwet eu queria compartilhar mas antes eu gos...,not_hate
1,2,gostei da pulseira e da educação dele nossaaa ...,not_hate
2,3,esses não irão ao encontro e beijing,not_hate
3,4,cadê eles está faltando,not_hate
4,5,e eu me questionando quem é esse blogueiro sem...,not_hate
...,...,...,...
91478,19277,estou sem presidente por quatro anos nãomerepr...,not_hate
91479,19278,somos todos iguais braços dados ou não quem sa...,not_hate
91480,19279,que deus nos proteja e cuidem um do outro somo...,not_hate
91481,19280,quem será que não sabe votar,not_hate


In [53]:
def balanceia_dados(df, verbose=0):
    # Esta funcao recebe um dataframe original e retorna um dataset
    # contendo um numero igual de noticias das categorias
    # por simplicidade, vou supor que são somente duas categorias.
    global categorias_desejadas
    df_balanceado = df.copy()
    
    # cria um dataframe para cada classe. Assim podemos remover o excesso
    df_F = df_balanceado.loc[df_balanceado.label == categorias_desejadas[0]]
    df_T = df_balanceado.loc[df_balanceado.label == categorias_desejadas[1]]

    num_amostras_F, num_amostras_T = df_F.shape[0], df_T.shape[0]
    num_amostras_a_remover = abs(num_amostras_F - num_amostras_T)
    
    # remove amostras da classe majoritaria
    if num_amostras_F > num_amostras_T:
        #print(df_F[:num_amostras_a_remover)
        df_F = df_F[num_amostras_a_remover:]
        
    elif num_amostras_T > num_amostras_F:
        #
        df_T = df_T[num_amostras_a_remover:]

    return pd.concat([df_F, df_T])

In [54]:
# Aplica função de balancear os dados
dataBC = balanceia_dados(df_filtrado)

# DATA BALANCEADO
trainDF = dataBC
trainDF.reset_index(drop=True, inplace=True) # reseta index
trainDF.shape

(16034, 3)

In [55]:
# convert output  hate =1 not_hate =0
trainDF.label = [1 if each == "hate" else 0 for each in trainDF.label] 

In [56]:
trainDF.groupby('label').count()

,Unnamed: 0,text
label,,
0,8017,8017
1,8017,8017


In [57]:
# Pega a média de caracteres dos tweets de toda a base
average_len = int(trainDF['text'].apply(lambda x: len(str(x))).mean())
average_len

71

In [58]:
quantidade_palavras_unicas = len( list ( trainDF['text'].str.split(' ', expand=True).stack().unique() ) )
quantidade_palavras_unicas

23378

## ShallowNet

In [59]:
# Salva os registros da coluna 'text' do dataframe em uma lista
text = []
trainDF['text'].apply(lambda x: text.append(x))
print(text)

['retwet a bahia em vai fazer anos sendo governada pelo pt eu não sei como o baiano e a bahia vem sup', 'k imagine se fosse atitude mental negativa se f idiota fechadocombolsolnaro', 'o general heleno é que nem o bolsonaro rosna muito late late late e não morde estou de saco cheio da passividade desse governo ainda não entenderam que guerra é guerra estamos em guerra c a pandemia c a política e c a mídia esquerdista', 'alan tem que ser muito cara de pau achar que está havendo uma ruptura nesse esquadrão de bandidos discurso contraditório só para causar mais caos vocês são ridículos', 'os canalhascomunistas caçarão a chapa presidencialno próximo dia assim será consolidado o golpe de estado comunista chinês essa ditadura chinesa será vezes pior do que a da venezuela depois me criticam por dizer que o exército está coptado', 'essa canalhice toda só ocorre em um país cujo exército permissivo se mantém inerte perante um golpe de estado materializado de usurpação dos poderes presidenciais ex

In [60]:
# Para que transformar os tuítes em matriz de números
tokenizer = Tokenizer(num_words=5000, split=' ')

tokenizer.fit_on_texts(text)
X = tokenizer.texts_to_sequences(text)  

# limitando o tamanho das sequências com base no tamanho máximo que especificamos na variável "max_sequence_length"
X = pad_sequences(X, maxlen=average_len)

In [61]:
Y = trainDF['label']

In [62]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, Y, test_size=0.3, random_state=2)

In [63]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(1000, activation="softmax")(input_layer)
     
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)

    classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
    
    return classifier 

# Modelo criado
model = create_model_architecture(train_x.shape[1])

# Modelo sendo treinado/executado
epochs = 1000
batch_size = 64
hist = model.fit(
      train_x, 
      train_y, 
      validation_data=(valid_x, valid_y),
      epochs=epochs,
      batch_size=batch_size, 
      shuffle=True,
      verbose=1)

model.save_weights("teste")    



Epoch 1/1000
176/176 [==============================] - 2s 6ms/step - loss: 0.6868 - accuracy: 0.5850 - val_loss: 0.6831 - val_accuracy: 0.5914
Epoch 2/1000
176/176 [==============================] - 1s 5ms/step - loss: 0.6777 - accuracy: 0.6031 - val_loss: 0.6715 - val_accuracy: 0.6136
Epoch 3/1000
176/176 [==============================] - 1s 5ms/step - loss: 0.6651 - accuracy: 0.6231 - val_loss: 0.6643 - val_accuracy: 0.6169
Epoch 4/1000
176/176 [==============================] - 1s 5ms/step - loss: 0.6596 - accuracy: 0.6194 - val_loss: 0.6654 - val_accuracy: 0.5926
Epoch 5/1000
176/176 [==============================] - 1s 4ms/step - loss: 0.6600 - accuracy: 0.6059 - val_loss: 0.6622 - val_accuracy: 0.5928
Epoch 6/1000
176/176 [==============================] - 1s 5ms/step - loss: 0.6558 - accuracy: 0.6086 - val_loss: 0.6587 - val_accuracy: 0.6001
Epoch 7/1000
176/176 [==============================] - 1s 5ms/step - loss: 0.6561 - accuracy: 0.6043 - val_loss: 0.6562 - val_accuracy:

KeyboardInterrupt: ignored

In [ ]:
# print(type(hist.history))
# print(hist.history.keys())
# print(hist.history)

In [ ]:
# Plot
plt.figure()
plt.plot(hist.history['loss'], lw=2.0, color='b', label='train')
plt.plot(hist.history['val_loss'], lw=2.0, color='r', label='val')
plt.title('Classificador de sentimentos')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy')
plt.legend(loc='upper right')
plt.show()

plt.figure()
plt.plot(hist.history['accuracy'], lw=2.0, color='b', label='train')
plt.plot(hist.history['val_accuracy'], lw=2.0, color='r', label='val')
plt.title('Classificador de sentimentos')
plt.xlabel('Epochs')
plt.ylabel('Acurácia')
plt.legend(loc='upper left')
plt.show()